In [1]:
%gui qt6

# Qt6 stuff
from PyQt6.QtWidgets import QApplication
from PyQt6 import QtWidgets
from PyQt6 import QtGui
from PyQt6 import QtCore
from PyQt6.QtCore import pyqtSignal

# Plotting stuff
import pyqtgraph as pg
import matplotlib.pyplot as plt
import cmasher as cmr

# Science stuff
import numpy as np
import pandas as pd
from spectres import spectres

# Astropy stuff
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.units.quantity import Quantity
from astropy.io.ascii import read as ascii_read


# General stuff
import logging
import sys
from pathlib import Path
from itertools import cycle

In [2]:
# zHunter stuff
from zhunter import DIRS
from zhunter import io
from zhunter.misc import set_up_linked_vb, get_vb_containing, convert_to_bins
from zhunter.colors import get_gradient
from zhunter.spectroscopic_system import SpecSystemModel, SpecSystem
from zhunter.MainGraphicsWidget import MainGraphicsWidget
from zhunter.data_handler import DataHandler

In [3]:
# astropalmerio stuff
import astropalmerio.io as io
import astropalmerio.spectra as sp
import astropalmerio.galaxies as gal
import astropalmerio.mc as mc
from astropalmerio.spectra import ergscm2AA
from astropalmerio.spectra import EmissionLine
from specutils import Spectrum1D

In [4]:
from zhunter.colors import COLORS
color_style = 'kraken9'
colors = COLORS[color_style]

In [5]:
log = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
                    format='%(asctime)s %(levelname)s [%(name)s] %(message)s')
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("PIL").setLevel(logging.WARNING)

In [6]:
# start qt event loop
_instance = QApplication.instance()
if not _instance:
    _instance = QApplication([])
app = _instance

In [19]:
win = pg.GraphicsLayoutWidget(show=True)
# To suppress qt.pointer.dispatch warning
win.viewport().setAttribute(
            QtCore.Qt.WidgetAttribute.WA_AcceptTouchEvents,
            False,
        )


ax = win.addPlot(col=0)
side_vb = win.addViewBox(col=1)
side_vb.setYLink(ax.vb)


In [21]:
# The fake data
N_data = 100
N_spat = 40
wvlg = np.linspace(300, 900, N_data)
spat = np.linspace(-10, 5, N_spat)
flux = np.ones((wvlg.shape[0], spat.shape[0])) * np.exp(-(spat[int(N_spat/2)]-spat)**2/0.1)
# add noise
flux += np.random.normal(0, 0.1, size=(wvlg.shape[0], spat.shape[0]))
# add an emission line
flux[:,int(N_spat/2)] += sp.gaussian_fct(wvlg, mean=656.28, stddev=0.5, amplitude=3)
flux = flux.T
unc = flux*0.1
wvlg_tell = wvlg
wvlg_sky_bkg = wvlg
tellurics = 1 - np.exp(-(wvlg-600)**2/(30)**2)
sky_bkg = np.exp(-(wvlg-800)**2/(10)**2)

data = DataHandler()
data.load_2D(wvlg*u.nm, spat*u.arcsec, flux*ergscm2AA, unc*ergscm2AA)
data.load_1D(wvlg*u.nm, np.sum(flux[17:23], axis=0)*ergscm2AA, np.sqrt(unc[17:23]**2).sum(axis=0)*ergscm2AA)


2023-03-22 10:58:29,261 DEBUG [zhunter.misc] Flux and uncertainty need to be rescaled (exponent: -1), rescaling them.
2023-03-22 10:58:29,265 DEBUG [zhunter.misc] Flux and uncertainty need to be rescaled (exponent: -1), rescaling them.


In [22]:
spec_2D = pg.ImageItem()
spec_2D.setImage(
            data["flux_2D_disp"].T.value,
            levels=(data["q025_2D"].value, data["q975_2D"].value),
        )
ax.addItem(spec_2D)

In [23]:
rect = QtCore.QRectF(
            data['wvlg_min'].value,      # lower edge of the first wvlg bin
            data['spat_min'].value,      # lower edge of the first spatial bin
            data['wvlg_span'].value,     # x-span of the rectangle
            data['spat_span'].value,  # y-span of the rectangle
        )

spec_2D.setRect(rect)

In [24]:
spec = pg.PlotCurveItem(
            np.zeros(2),
            np.zeros(1),
            stepMode="center",
    rotation=-90
        )

In [25]:
y_dist = np.median(data["flux_2D_disp"], axis=1)

In [28]:
spec.setData(-data["spat_bins_disp"].value, y_dist.value)
side_vb.addItem(spec)

In [27]:
spec.setRotation(-90)

In [15]:
ax.hideAxis('bottom')

In [7]:
cbb = QtWidgets.QComboBox()
cbb

In [8]:
cbb.show()

In [12]:
cbb.addItems(['1', '2', 'test'])

In [22]:
np.random.normal(100)

100.72917826215429

In [23]:
bins = np.linspace(-10, 10, int(20/0.25)+1)
x = np.histogram(np.random.normal(size=100), bins, density=True)
x

(array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.04, 0.04, 0.04, 0.04,
        0.08, 0.12, 0.2 , 0.12, 0.44, 0.44, 0.2 , 0.4 , 0.44, 0.48, 0.44,
        0.12, 0.24, 0.08, 0.  , 0.  , 0.  , 0.04, 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  ]),
 array([-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
         -8.  ,  -7.75,  -7.5 ,  -7.25,  -7.  ,  -6.75,  -6.5 ,  -6.25,
         -6.  ,  -5.75,  -5.5 ,  -5.25,  -5.  ,  -4.75,  -4.5 ,  -4.25,
         -4.  ,  -3.75,  -3.5 ,  -3.25,  -3.  ,  -2.75,  -2.5 ,  -2.25,
         -2.  ,  -1.75,  -1.5 ,  -1.25,  -1.  ,  -0.75,  -0.5 ,  -0.25,
          0.  ,   0.25,   0.5 ,   0.75,   1.  ,   1.25,   1.5 ,   1.75,
          2.  ,   2.25

In [37]:
zz = pg.InfiniteLine(2)

In [25]:
import numpy as np
import astropy.units as u
from astropalmerio.galaxies import star_formation_rate
M1600 = -18.2
f_0 = 3.63078*10**9 * u.uJy
L_nu = 4*np.pi * f_0 * (10 * u.parsec)**2  * 10**(-0.4*M1600)
print("Lnu :", L_nu.to(u.Unit('erg/s/Hz')))
SFR = 1.3 * 10**(-28) * L_nu.to(u.Unit('erg/s/Hz'))
print("SFR : ", SFR.value)
star_formation_rate(2.3e-18,z=3.3467, normalization='Chabrier')

2023-03-27 15:56:02,241 WARNING [astropy] UnitsWarning: 'erg/s/Hz' contains multiple slashes, which is discouraged by the FITS standard
Lnu : 8.277722991913292e+27 erg / (Hz s)
SFR :  1.076103988948728
2023-03-27 15:56:02,244 INFO [astropalmerio.galaxies.properties] Using Planck 2018 cosmology from astropy : FlatLambdaCDM(name="Planck18", H0=67.66 km / (Mpc s), Om0=0.30966, Tcmb0=2.7255 K, Neff=3.046, m_nu=[0.   0.   0.06] eV, Ob0=0.04897)


1.113245130640815

In [41]:
def aa(**args):
    mean_guess = 10
    means = args.get("mean", mean_guess)
    print(f"got {means}")
    print("also", *args)

In [42]:
args = {}
if zz.getPos()[0] != 0:
    args['mean'] = zz.getPos()[0]
    aa(**args)


got 2
also mean


In [56]:
x = {
    'test':3,
    'ok_fit':2,
    'blo_fit':'haha',
    'zio_fit':9,
}

for k in list(x.keys()):
    if '_fit' in k:
        x.pop(k)

In [57]:
x

{'test': 3}

In [22]:
dspat_1D = data['spat_1D_disp'][1]-data['spat_1D_disp'][0]
dspat_1D

<Quantity 0.25641026 arcsec>

In [9]:
brush = pg.mkBrush(colors["continuum"]+'30')
hoverbrush = pg.mkBrush(colors["continuum"]+'60')
cr.setBrush(brush)
cr.setHoverBrush(hoverbrush)

In [10]:
li = pg.LinearRegionItem()

qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=589.975,393.389 gbl=589.975,393.389 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-589.975,-393.389 last=-589.975,-393.389 Δ 589.975,393.389) : no target window


In [97]:
li.

ValueError: list.remove(x): x not in list

In [78]:
x2

['1', '2', '3']